In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [2]:
# svg, png. no inline pdf.

In [3]:
import matplotlib as mpl
import matplotlib.pyplot as plt
#import seaborn as sns

In [4]:
import numpy as np
import scipy
import scipy.stats
import pandas

In [5]:
import sklearn

In [6]:
import collections
import cPickle as pickle
import unicodecsv as csv
import json
import re

In [7]:
import analysis_tools

In [8]:
np.seterr(divide='raise', over='raise', under='raise', invalid='raise');

# Data preparation

Let's load the cities. We'll filter out any groups that weren't 'active' during the period of study (2013-2014, inclusive). Active groups are those that have had an event with at least two attendees.

In [10]:
all_city2groups = analysis_tools.get_city2groups('GB')
print "number of groups before filtering:", sum(len(groups) for groups in all_city2groups.itervalues())

 number of groups before filtering: 1588


In [11]:
def is_active(grp):
    for event in grp['events_in_window']:
        if len(event['attendee_ids']) >= 2:
            return True
    return False
        
for city_ident in all_city2groups.iterkeys():
    all_city2groups[city_ident] = filter(is_active, all_city2groups[city_ident])
    
print "number of groups after filtering:", sum(len(groups) for groups in all_city2groups.itervalues())

number of groups after filtering: 938


How many usable groups do we have per city?

In [14]:
group_counts = collections.Counter({city: len(groups) for city, groups in all_city2groups.iteritems()})

for indx, (k, v) in enumerate(group_counts.most_common(5)):
    print "%-10s%-10s%s" % (indx+1, v, k)
print "..."

1         545       GB::London
2         36        GB::Manchester
3         35        GB::Bristol
4         32        GB::Cambridge
5         25        GB::Edinburgh
...


In [18]:
min_num_groups = 2
keep_cities = [city for city, count in group_counts.iteritems() if count >= min_num_groups]

print "%d cities with at least %s groups:" % (len(keep_cities), min_num_groups)
print ', '.join(keep_cities)

city2groups = {city: all_city2groups[city] for city in keep_cities}  # all cities for study

43 cities with at least 2 groups:
GB::Milton Keynes, GB::Cardiff, GB::Leicester, GB::Coventry, GB::Southampton, GB::Hull, GB::London, GB::Norwich, GB::Bath, GB::Belfast, GB::Birmingham, GB::Temple Meads, GB::Guildford, GB::Isleworth, GB::Poole, GB::Colchester, GB::Canterbury, GB::Swindon, GB::Nottingham, GB::Northampton, GB::Reading, GB::Plymouth, GB::Bristol, GB::Glasgow, GB::Manchester, GB::Cambridge, GB::Hove, GB::Liverpool, GB::Ipswich, GB::Sheffield, GB::Exeter, GB::Warrington, GB::Tunbridge Wells, GB::Edinburgh, GB::Swansea, GB::Marlow, GB::Cheltenham, GB::Oxford, GB::Leeds, GB::Brighton, GB::Newcastle Upon Tyne, GB::Luton, GB::Peterborough


In [20]:
print "number of groups among cities retained for study:", sum(len(groups) for groups in city2groups.itervalues())

number of groups among cities retained for study: 868


# PCA

In [23]:
topic_counts = collections.Counter()

for city, groups in city2groups.iteritems():
    for group in groups:
        for topic_dict in group['topics']:
            topic_name = topic_dict['name']
            topic_counts[topic_name]